# Part 1 - Scraping neighborhood data from Wikipedia

In [93]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import folium

In [94]:
# Scraping the wikipedia website
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
soup = BeautifulSoup(response.content)

# Converting to human readable content
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df = df[0].to_json()
df[0:1000]

'{"Postcode":{"0":"M1A","1":"M2A","2":"M3A","3":"M4A","4":"M5A","5":"M6A","6":"M6A","7":"M7A","8":"M8A","9":"M9A","10":"M1B","11":"M1B","12":"M2B","13":"M3B","14":"M4B","15":"M4B","16":"M5B","17":"M5B","18":"M6B","19":"M7B","20":"M8B","21":"M9B","22":"M9B","23":"M9B","24":"M9B","25":"M9B","26":"M1C","27":"M1C","28":"M1C","29":"M2C","30":"M3C","31":"M3C","32":"M4C","33":"M5C","34":"M6C","35":"M7C","36":"M8C","37":"M9C","38":"M9C","39":"M9C","40":"M9C","41":"M1E","42":"M1E","43":"M1E","44":"M2E","45":"M3E","46":"M4E","47":"M5E","48":"M6E","49":"M7E","50":"M8E","51":"M9E","52":"M1G","53":"M2G","54":"M3G","55":"M4G","56":"M5G","57":"M6G","58":"M7G","59":"M8G","60":"M9G","61":"M1H","62":"M2H","63":"M3H","64":"M3H","65":"M3H","66":"M4H","67":"M5H","68":"M5H","69":"M5H","70":"M6H","71":"M6H","72":"M7H","73":"M8H","74":"M9H","75":"M1J","76":"M2J","77":"M2J","78":"M2J","79":"M3J","80":"M3J","81":"M4J","82":"M5J","83":"M5J","84":"M5J","85":"M6J","86":"M6J","87":"M7J","88":"M8J","89":"M9J","90":"

## Converting and cleaning the dataframe

In [95]:
# Converting to DataFrame
data = json.loads(df)
data = pd.DataFrame(data)
data.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [96]:
# Dropping the empty Borough rows
data = data[data.Borough != 'Not assigned'].reset_index(drop=True)
data.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [97]:
# Checking for neighborhoods which are 'Not Assigned'
data[data.Neighborhood == 'Not assigned']

,Postcode,Borough,Neighborhood
5,M7A,Queen's Park,Not assigned


In [98]:
# Assigning missing neighborhood name to it's borough
data.loc[[5], ['Neighborhood']] = "Queen's Park"

In [99]:
# Merging the rows by postcode name
data = data.groupby(['Postcode', 'Borough'], as_index=False)['Neighborhood'].apply(', '.join).reset_index()

In [100]:
# Renaming the column
data = data.rename(columns={0: 'Neighborhood'})
data.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [101]:
data.shape

(103, 3)

# Part 2 - getting the latitude and longitude

In [102]:
# Read the coordinate data
loc_data = pd.read_csv('Geospatial_Coordinates.csv')
loc_data = loc_data.rename(columns={'Postal Code': 'Postcode'})
loc_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [103]:
# Merging the dataframes by Postcode
df = pd.merge(data, loc_data, how='outer', on='Postcode')
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [104]:
df.shape

(103, 5)

# Part 3 - exploring and clustering neighborhoods in Toronto

In [221]:
from geopy.geocoders import Nominatim

# Get the coordinates of Toronto

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="neigh_toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [106]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [107]:
# Get Foursquare credentials

CLIENT_ID = 'JLPFMODERBG0EMQHWGJ5KIZPLE3T2FCATEDLKSQZHOUM0WUU' # your Foursquare ID
CLIENT_SECRET = '1PALU5LZP0FNWBFJUXTHHDTHDGLALBZRVE0Y1CLQ20C0REJF' # your Foursquare Secret
VERSION = '20191231' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JLPFMODERBG0EMQHWGJ5KIZPLE3T2FCATEDLKSQZHOUM0WUU
CLIENT_SECRET:1PALU5LZP0FNWBFJUXTHHDTHDGLALBZRVE0Y1CLQ20C0REJF


### Define function that returns nearby venues for a given locations

In [108]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    """Function that returns nearby venues of a given locations"""
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [223]:
# get the venues for given coordinates and name

"""
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

"""

"\ntoronto_venues = getNearbyVenues(names=df['Neighborhood'],\n                                   latitudes=df['Latitude'],\n                                   longitudes=df['Longitude']\n                                  )\n\n"

## Read previously saved downloaded data to work on the same results

In [110]:
toronto_venues = pd.read_csv('toronto_venues.csv')

In [111]:
# size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2233, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


# Save dataframe to file to work on the same results all the time

with open('toronto_venues.csv', 'w') as file:
    toronto_venues.to_csv(file, encoding='utf-8', index=False)
    

In [112]:
# how many venues were returned in a neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10
...,...,...,...,...,...,...
Willowdale West,6,6,6,6,6,6
Woburn,4,4,4,4,4,4
"Woodbine Gardens, Parkview Hill",11,11,11,11,11,11


In [113]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


In [225]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# get the index of neighborhood column
cols = toronto_onehot.columns.to_list()
print(cols.index('Neighborhood'))

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[194]] + list(toronto_onehot.columns[0:194]) + list(toronto_onehot.columns[195:275])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

194
(2233, 275)


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [226]:
# count the mean values of venues for each neighborhood

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
95,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
96,"Woodbine Gardens, Parkview Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
97,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0


### Check if number of Neighborhoods is the same

In [227]:
toronto_grouped.shape

(99, 275)

In [228]:
len((toronto_venues['Neighborhood'].unique()))

99

### Define a function and count the most common venues

In [118]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [214]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Restaurant,Sushi Restaurant,Burger Joint,Asian Restaurant,Cosmetics Shop
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Discount Store,Sandwich Place,Beer Store,Japanese Restaurant,Pharmacy,Fast Food Restaurant,Dumpling Restaurant
4,"Alderwood, Long Branch",Pizza Place,Pub,Pharmacy,Sandwich Place,Dance Studio,Athletics & Sports,Skating Rink,Coffee Shop,Gym,Comic Shop


## Clustering Neighborhoods

In [215]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0, n_init=15).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [216]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.shape)

toronto_merged.head() # check the last columns!

(103, 16)


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,Moving Target,Bar,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Rental Car Location,Intersection,Electronics Store,Mexican Restaurant,Breakfast Spot,Pizza Place,Medical Center,Dim Sum Restaurant,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Indian Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Gas Station,Bakery,Fried Chicken Joint,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Bank,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant


In [217]:
# Drop a few rows where data was not downloaded

toronto_merged = toronto_merged.dropna()
toronto_merged.shape

(100, 16)

In [218]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

In [219]:
# create one dataframe from all clusters

toronto_clustered = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


for n in range(1, 7):
    
    toronto_temp = toronto_merged.loc[toronto_merged['Cluster Labels'] == n, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
    toronto_clustered = toronto_clustered.append(toronto_temp, ignore_index=True)
    
toronto_clustered.reset_index(drop=True)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",0.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Falafel Restaurant
1,"Highland Creek, Rouge Hill, Port Union",0.0,Moving Target,Bar,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,"Guildwood, Morningside, West Hill",0.0,Rental Car Location,Intersection,Electronics Store,Mexican Restaurant,Breakfast Spot,Pizza Place,Medical Center,Dim Sum Restaurant,Diner,Discount Store
3,Woburn,0.0,Coffee Shop,Korean Restaurant,Indian Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,Cedarbrae,0.0,Gas Station,Bakery,Fried Chicken Joint,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Bank,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Caledonia-Fairbanks,2.0,Park,Market,Fast Food Restaurant,Women's Store,Airport,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
96,"Downsview, North Park, Upwood Park",2.0,Park,Basketball Court,Bakery,Construction & Landscaping,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
97,"The Kingsway, Montgomery Road, Old Mill North",2.0,Park,Smoke Shop,River,Pool,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
98,Weston,2.0,Convenience Store,Park,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store


In [220]:
# Count each elements in a cluster

toronto_clustered.groupby('Cluster Labels')['Neighborhood'].count()

Cluster Labels
0.0    84
1.0     2
2.0    14
Name: Neighborhood, dtype: int64

## As we can see the algorithm in this set clusters the vanues based on the urban and green areas, each venue set makes for different clusters